<a href="https://colab.research.google.com/github/colbrydi/Scientific_Image_Understanding/blob/master/05-Registration-pre-class-assignment.ipynb"><img src="https://camo.githubusercontent.com/52feade06f2fecbf006889a904d221e6a730c194/68747470733a2f2f636f6c61622e72657365617263682e676f6f676c652e636f6d2f6173736574732f636f6c61622d62616467652e737667"></a>

# Pre-Class Assignment: Image Registration



# Goals for today's pre-class assignment 

1. [Image Registration](#Image-Registration)
2. [Basic Rigid Transforms](#Basic-Rigid-Transforms)
3. [ Affine Transforms](#-Affine-Transforms)
4. [Projective Transformations](#Projective-Transformations)
5. [Homography](#Homography)





---
<a name=Image-Registration></a>
# 1. Image Registration



Image registration is the process of transforming different sets of image data into a common coordinate system.  This is like moving one image on top of another image.  A reasonable overview for image Registration can be found here;

https://en.wikipedia.org/wiki/Image_registration

Consider the following example of two pictures of Beaumont Tower at Michigan State university:

In [ ]:
# Read data for this assignment
%matplotlib inline 
import matplotlib.pyplot as plt
from urllib.request import urlopen, urlretrieve
from imageio import imread, imsave

url1 = 'http://res.cloudinary.com/miles-extranet-dev/image/upload/ar_16:9,c_fill,w_1000,g_face,q_50/Michigan/migration_photos/G21696/G21696-msubeaumonttower01.jpg'
file1='Tower1.jpeg'
urlretrieve(url1, file1)
im1 = imread(file1)
    

url2 = 'https://research.msu.edu/wp-content/uploads/2019/11/beaumont-winter.jpg'
file2 = 'Tower2.jpeg'
urlretrieve(url2, file2)
im2= imread(file2)
    

f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(im1)
ax2.imshow(im2)



---
<a name=Basic-Rigid-Transforms></a>
# 2. Basic Rigid Transforms




**&#9989;  DO THIS:** Use the following code and sliders to register the second image onto the first image. Try to find the best fit so that the towers line up exactly.  

The sliders will let you change the scale (s), x translation (tx), y-translation (ty), and rotation angle (angle).  You can also adjust the alpha measure to help "see though" one image into the other.

In [ ]:
from __future__ import division
import matplotlib.pyplot as plt
import numpy as np
from skimage import transform
from ipywidgets import interact, fixed
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

im = im1
def affine_image(im1, im2, s=1,tx=0,ty=0, angle=0, alpha=0.5):
    theta = -angle/180  * np.pi
    
    dx = tx*im.shape[1]
    dy = ty*im.shape[0]
    S = np.matrix([[1/s,0,0], [0,1/s,0], [0,0,1]])
    T2 = np.matrix([[1,0,im.shape[1]/2], [0,1,im.shape[0]/2], [0,0,1]])
    T1 = np.matrix([[1,0,-im.shape[1]/2-dx], [0,1,-im.shape[0]/2-dy], [0,0,1]])
    R = np.matrix([[np.cos(theta),-np.sin(theta),0],[np.sin(theta), np.cos(theta),0],[0,0,1]])

    T = T2*S*R*T1;
    img = transform.warp(im, T);
    plt.imshow(im2);
    plt.imshow(img, alpha=alpha);

    plt.show();
    
interact(affine_image, 
         im1=fixed(im1), 
         im2=fixed(im2), 
         s=(0.001,5), 
         tx=(-1.0,1.0), 
         ty=(-1,1,0.1), 
         angle=(-180,180), 
         alpha=(0.0,1.0)); 



---
<a name=-Affine-Transforms></a>
# 3.  Affine Transforms


The above example is for an Rigid body transform.  However, more complex transformations are possible.  Watch the following video to learn more about Affine Transforms.

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("il6Z5LCykZk",width=640,height=360)




---
<a name=Projective-Transformations></a>
# 4. Projective Transformations




Projective transforms add additional degrees of freedom and can "skew" an image. Watch the following video on Projective Transforms.  

In [ ]:
from IPython.display import YouTubeVideo
YouTubeVideo("uyYKPUZg3og",width=640,height=360)


---
<a name=Homography></a>
# 5. Homography

The Homography operations calculates a projective transformation using a set of points on one plain mapped to a similar set of points on a different plain. Since images often represent plains this turns out to be a useful registration method in image analysis. More information about the math can be found here:

https://en.wikipedia.org/wiki/Homography_(computer_vision)

and

http://people.scs.carleton.ca/~c_shu/Courses/comp4900d/notes/homography.pdf

Consider the following example code from a psychological study in game theory.  The researcher would like to have a computer watch a game between two checkers players and analysis their moves.  To do this they need to register the checkboards to the same grid.  The code below uses the ```skimage.transform.ProjectiveTransform``` which calculates the transform using homography.

In [ ]:
#The following code snip-it downloads a file from internet and saves it to your local directory.
from urllib.request import urlopen, urlretrieve
import scipy.misc as misc

url = 'https://goo.gl/j2SFnL'
file1 = 'Checkers.png'
urlretrieve(url, file1);
im = imread(file1)

#Points in source image coordinate system
src = np.array([[156, 197],[284, 181],[407, 177],[172, 296],[318, 275],[452, 264],[190, 418],[359, 387],[507, 371]])

In [ ]:
#Calculate Transform
from skimage import transform

width=1000

#Points in desitnation coordinate system
dst = np.array([[0, width/2, width, 0, width/2, width, 0, width/2, width,], 
                [0, 0, 0, width/2, width/2, width/2, width, width, width]]).T

#Calculate projective transform from the source to the destination
tform = transform.ProjectiveTransform()
tform.estimate(dst, src)
im2 = transform.warp(im, tform, output_shape=(width,width)) 


In [ ]:
import sympy as sym
sym
sym.Matrix(tform.params)

In [ ]:
#show original image next to transformed image

f, (ax1, ax2) = plt.subplots(1, 2,figsize=(20,10))
ax1.imshow(im)
ax1.scatter(src[:,0],src[:,1])
#Add numbers
for i in range(dst.shape[0]):
    ax1.annotate(str(i+1), (src[i,0]+20,src[i,1]), color='white');
ax1.set_title('Source Plain')


ax2.imshow(im2)
ax2.scatter(dst[:,0],dst[:,1])
#Add numbers
for i in range(dst.shape[0]):
    ax2.annotate(str(i+1), (dst[i,0]+20,dst[i,1]));
ax2.set_title('Destination Plain')
ax2.axis('equal');

---

Written by Dr. Dirk Colbry, Michigan State University
<a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc/4.0/">Creative Commons Attribution-NonCommercial 4.0 International License</a>.

---